In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

In [2]:
f = open('Datasets/Text/RT.txt','r',encoding="utf8")
total_str = f.read()
f.close()

In [3]:
vocab = sorted(list(set(list(total_str))))
voc_sz = len(vocab)

In [15]:
print(''.join(vocab))
print(len(vocab))

	
 !"'(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
77


In [5]:
itoa = {}
atoi = {}
for i,ele in enumerate(vocab):
    atoi[ele] = i
    itoa[i] = ele

In [6]:
Xdata = []
for e in total_str:
    Xdata.append(atoi[e])
Xdata = torch.tensor(Xdata)

In [7]:
Xt = []
Yt = []
for i in range(len(Xdata)-8):
    Xt.append(Xdata[i:i+8])
    Yt.append(Xdata[i+8])
Xt = torch.stack(Xt)
Yt = torch.stack(Yt)
# Yt = F.one_hot(Yt)


In [8]:
Xt[0],Yt[0] # nice data arranged

(tensor([ 1,  1, 30, 39, 42, 29, 47, 39]), tensor(42))

In [9]:
xtrain = Xt[:int(0.9*len(Xt))]/voc_sz
ytrain = Yt[:int(0.9*len(Yt))]
xtest = Xt[int(0.9*len(Xt)):]/voc_sz
ytest = Yt[int(0.9*len(Yt)):]

In [10]:
# variables defining : 

head_sz = 32
B,T,C = 4,8,32  # attention but yeah
n_fl = 100
n_ml = 256
# n_ol = 32
n_octms = len(vocab)
l_r = 0.0001
ex_len = 8
bat_sz = 4
epoch = 10

In [11]:
# lets make it my net

In [26]:
modely = nn.Sequential(
    nn.Linear(ex_len,n_fl),
    nn.ReLU(),
    nn.Linear(n_fl,n_ml), # will give 10 outputs (probs)
    # nn.Dropout(0.0),
    nn.Linear(n_ml,voc_sz),
    # nn.Softmax(),
)

ix = (torch.randperm(len(xtrain)-bat_sz))

In [27]:
def accuracy():
    k,d=len(xtest),len(xtest)
    for n in range(len(xtest)):
        omt = ((modely(xtest[n].float()))).tolist()
        # print(omt.index(max(omt)),ytest[n].item())

        k -= (omt.index(max(omt))==ytest[n].item())
    accu = (((d-k)/d) *100)
    return("Accuracy = " + str(accu))

In [28]:
optimizer = torch.optim.SGD(modely.parameters(), lr=0.001)

In [29]:
for e in range(epoch):
    ahead = 0
    for i in ix[ahead :ahead + 100000]:
        xt = xtrain[i:i+bat_sz]
        yt = ytrain[i:i+bat_sz]
        X_train = xt.view([bat_sz,ex_len]) #[4,784]
        Y_train = F.one_hot(yt,voc_sz).float()
        # print(Y_train.shape)
        # break
        t = modely(X_train.float())
        loss = F.cross_entropy(t,Y_train.float())
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()
        
        # print(loss.item())
    # break
    print(accuracy())
    if(e == int(0.8*epoch)):
        optimizer = torch.optim.SGD(modely.parameters(), lr=0.0001)
    ahead +=100000

Accuracy = 21.29587902458565
Accuracy = 21.549997496369734
Accuracy = 22.110810675479446
Accuracy = 22.08890391067047


KeyboardInterrupt: 

EXTRA
DOWN

In [22]:
# attention
@torch.no_grad()
def self_attention(x):  

    B,T,C = x.shape
    # B = 4  # block size
    # T = 8  # token (number of tokens in one example)
    # C = 32 # embedding space vector of the each one token
    # x = torch.randn(B,T,C) # example sequence
    # x -> this is how our one batch should look like here 


    # head_sz = 32# this is the splits we do of each token seqence to parellelize the sequence gen

    key = nn.Linear(C,head_sz,bias=False) # this is to make to complexities truly match  
    query =  nn.Linear(C,head_sz,bias=False) 
    value =  nn.Linear(C,head_sz,bias=False) 
    k = key(x)  # [B,T,head_sz]
    q = query(x)  # [B,T,head_sz]
    # this is -- x @ key --  then 
    # upcoming wei is the main token to token connection matrix
    wei = q @ k.transpose(-2,-1)
    # wei[0]  # TxT

    tril = torch.tril(torch.ones(T, T))  # makes a lower triangular matrix of TxT
    wei = wei.masked_fill(tril == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)

    v = value(x)
    out = wei @ v
    # out[0].shape
    return out

# basically this is what makes up the new and improved input to the neural net

In [12]:
len(Yt)==len(Xt)

True

In [38]:

# making the mlp for taking the input
Model = nn.Sequential(
    nn.Linear(head_sz,n_fl),
    nn.Linear(n_fl,n_ml),
    nn.Linear(n_ml,n_ol),
    # nn.Dropout(),
    nn.Linear(n_ol,n_octms),
    nn.Softmax()
)




In [39]:
# lets first apply the embedding space to our data

embeder  = nn.Embedding(voc_sz,C) 
optimizer = torch.optim.SGD(Model.parameters(), lr=l_r)

In [42]:
# a.shape  : [1597676, 8, 32]
# now we make the things on a row
examples = (Xt.shape)[0]
n_iters = 200

for i in range(n_iters):
    t = random.randint(0,examples-B)
    ex = Xt[t:t+B]  # getting training examples
    ey = Yt[t:t+B]
    with torch.no_grad():
        ex = embeder(ex)
    slfa_ex = self_attention(ex)  # this would return [B,T,head_sz]  (head_sz : C)here
    # we get the attention values to add to the existing ex
    print(ex[0][0])
    ex += slfa_ex
    print(ex[0][0])
    break
    # final input made 

    logits = Model(ex)  # this is [4,8,77] i.e in a batch of 4 every token ke baad wale token ka prediction is available
    # print(logits[0],ey[0])
    loss = F.cross_entropy(logits, ey)  # inputs and outputs in batches
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # if n_iters == 20:
    #     break


    print(loss.item())


tensor([ 0.0494,  0.1003, -2.4102, -0.0692,  0.1053, -0.3065, -0.5829, -0.3122,
         0.0683,  0.8160,  0.9703, -2.0326, -1.4490, -0.5732, -1.5804, -1.7114,
        -1.0672, -0.2550, -1.0839, -1.3513,  0.7072, -0.4863, -0.4208,  0.2644,
         0.3307, -1.0344, -0.1496,  0.3963,  0.7138,  0.7783, -0.7913,  1.0514])
tensor([-0.5190,  0.8751, -2.9042,  0.2317,  1.0372, -0.3224, -2.2645, -0.0314,
        -0.3553,  0.2634,  0.9047, -2.5852, -1.1306, -1.5207, -1.0926, -1.9229,
        -1.4651, -0.1304, -0.5489, -1.6495,  1.0376, -0.4842, -0.1917,  0.7149,
         0.4716, -2.2820, -0.4293,  0.6612,  0.2918,  0.8176,  0.0039,  1.1029])


In [ ]:
Model.prediction()

In [ ]:
# extra

In [ ]:
class Reshaper(nn.Module):
    def __init__(self,n):
        self.n = n
        super(Reshaper, self).__init__()

    def forward(self, x):
        B, T, C = x.shape
        x = x.view(B, T//self.n, C*self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x
        return self.out

blk_sz = 8
emb_sz = 16
n_fl = 200
voc_sz = len(vocab)
inp_sz = 8
Model = nn.Sequential(
    nn.Embedding(voc_sz,emb_sz),     #  Gets the input ([10000,8,16])
    Reshaper(2),  # becomes [10000,4,32]  # [inp_sz,blk_sz//2,emb_sz*2]
    nn.Linear(emb_sz*2,n_fl),       # takes [32,200]  becomes [10000,4,200]
    nn.Tanh(),
    Reshaper(2),  # becomes [10000,2,400]   # [inp_sz,blk_sz//4,n_fl*2]
    nn.Linear(n_fl*2,n_fl),    # becomes  [10000,2,200]
    nn.Tanh(),
    Reshaper(2),  # becomes [10000,400]  # [inp_sz,n_fl*2]
    nn.Linear(n_fl*2,voc_sz),    # become [400,voc_sz]
    nn.Softmax(),
)


for x in range(10):
    for x in range(20000):
        ix = torch.randint(0, X_data.shape[0], (8,))
        Xb, Yb = X_data[ix], Y_data[ix] # batch X,Y
        # print(Xb)
        # break
        logits = Model(Xb)
        #print(logits , Yb.squeeze(dim = 1))
        loss = F.cross_entropy(logits, Yb.squeeze(dim = 1))
        Model.zero_grad()
        optimizer = torch.optim.SGD(Model.parameters(), lr=0.01)
        loss.backward()

        optimizer.step()

    print(loss)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

f = open('Datasets/Text/RT.txt','r',encoding="utf8")
text = f.read()
f.close()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))